In [8]:
#%%
print("ok")

ok


In [9]:
allNormalText = open(r"./data/data-normal.txt", 'r', encoding='utf-8').read()
allNormalTextArray = allNormalText.split('\n')
print(len(allNormalTextArray))

allAdText = open(r"./data/data-pssisterad.txt", 'r', encoding='utf-8').read()
allAdTextArray = allAdText.split('\n')
print(len(allAdTextArray))

1105
374


In [10]:
from gensim.models import Word2Vec
from model import AdPredictor
adp = AdPredictor()
splitNormalWords = [list(adp.splitWords(ad, False)) for ad in allNormalTextArray+allAdTextArray]
print(len(splitNormalWords))
splitNormalWords[0:10]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Kenvix\AppData\Local\Temp\jieba.cache
Loading model cost 1.048 seconds.
Prefix dict has been built successfully.
1479


[['没有', '上次', '学长', '学姐', '教', '几节课'],
 ['学姐', '交', '一下'],
 ['认识', '学姐', '保研'],
 ['党明', '学姐', '真的', '强'],
 ['想不到', '没有', '党明', '学姐', '做', '不到'],
 ['党明', '学姐'],
 ['党明', '学姐', '献上'],
 ['学姐'],
 ['我见', '证明', '党明', '学姐', '非常', '漂亮', '女孩子'],
 ['开机', '慢点', '还好']]

In [11]:
from gensim import corpora
# 赋给语料库中每个词(不重复的词)一个整数id
dictionary = corpora.Dictionary(splitNormalWords)
new_corpus = [dictionary.doc2bow(text) for text in splitNormalWords]
print(dictionary.get("学姐"))
print(new_corpus[0:10])

None
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(2, 1), (6, 1), (7, 1)], [(2, 1), (8, 1), (9, 1)], [(2, 1), (10, 1), (11, 1), (12, 1)], [(2, 1), (5, 1), (10, 1), (13, 1), (14, 1), (15, 1)], [(2, 1), (10, 1)], [(2, 1), (10, 1), (16, 1)], [(2, 1)], [(2, 1), (10, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(22, 1), (23, 1), (24, 1)]]


In [12]:
from gensim import models
tfidf = models.TfidfModel(new_corpus)
tfidf.save("tfidf.gsm")

# 使用这个训练好的模型得到单词的tfidf值
tfidf_vec = []
tfidf_vocabs = []
for i in range(len(splitNormalWords)):
    string_bow = dictionary.doc2bow(splitNormalWords[i])
    string_tfidf = tfidf[string_bow]
    tfidf_vec.append(string_tfidf)

print(tfidf_vec[0:5])
tfidf_vocabs_word_to_id = dictionary.token2id
print(tfidf_vocabs_word_to_id.get("学姐"))
print(tfidf_vocabs_word_to_id.get("大佬"))
# print(tfidf_vocabs)
tfidf_vocabs = {value:key for (key, value) in tfidf_vocabs_word_to_id.items()}
print(tfidf_vocabs[0])

[[(0, 0.43486697887608194), (1, 0.5119171361324011), (2, 0.31321233740410154), (3, 0.41469066267165816), (4, 0.46330389940202965), (5, 0.2532009857936268)], [(2, 0.5465089583938454), (6, 0.4653393010174748), (7, 0.6962666826179764)], [(2, 0.49641058193340654), (8, 0.6572436925228606), (9, 0.567104278579744)], [(2, 0.4157386673474663), (10, 0.5296619173467123), (11, 0.6149609153670653), (12, 0.4104176974201508)], [(2, 0.35642683093967287), (5, 0.2881356006126524), (10, 0.45409708910125546), (13, 0.37192254000856334), (14, 0.3256468748375844), (15, 0.5825473033649264)]]
2
153
上次


In [13]:
from collections import OrderedDict
import numpy as np

def tfidf_wtd_avg_word_vectors(words,tfidf_vector,tfidf_vocabulary,model,num_features):
    #print("tfidf_vector", tfidf_vector)
    word_tfidfs = []
    word_tfidf_map = OrderedDict()

    for (key, proba) in tfidf_vector:
        word_tfidfs.append(proba)
        word_tfidf_map[tfidf_vocabulary[key]] = proba
    
    # print(word_tfidfs)
    # print(word_tfidf_map)
    
    feature_vector=np.zeros((num_features,),dtype='float64')
    vocabulary=set(model.wv.index2word)
    wts=0
    for word in words:
        if word in vocabulary:
            word_vector=model[word]
            weighted_word_vector=word_tfidf_map[word]*word_vector
            wts=wts+word_tfidf_map[word]
            feature_vector=np.add(feature_vector,weighted_word_vector)
    if wts:
        feature_vector=np.divide(feature_vector,wts)
    return feature_vector

def tfidf_weighted_averaged_word_vectorizer(corpus,tfidf_vectors,tfidf_vocabulary,model,num_features):
    docs_tfidfs=[(doc,doc_tfidf) for doc,doc_tfidf in zip(corpus,tfidf_vectors)]
    features=[tfidf_wtd_avg_word_vectors(tokenized_sentence,tfidf,tfidf_vocabulary,model,num_features) for tokenized_sentence,tfidf in docs_tfidfs]
    return np.array(features)

# vec = Word2Vec(splitNormalWords,size=10,window=10,min_count=2,sample=1e-3)
word2vec = Word2Vec(splitNormalWords, min_count=3, sample=1e-3, size=10, window=10)
vector = tfidf_weighted_averaged_word_vectorizer(corpus=splitNormalWords, tfidf_vectors=tfidf_vec, tfidf_vocabulary=tfidf_vocabs,model=word2vec,num_features=10)
vector = np.array(vector)
# vec.build_vocab(splitNormalWords)
# vec.train(splitNormalWords, total_examples=len(splitNormalWords), epochs=vec.epochs)
# # vec.save("vec.gsm")
# vec
print(len(vector))
word2vec.save("word2vec.gsm")
vector[0:100]

1479


array([[ 0.03726877, -0.0260108 ,  0.00364225, -0.00761493,  0.0261371 ,
         0.02732867,  0.05900647,  0.0104771 , -0.03713089, -0.00172936],
       [ 0.05519021, -0.03163468,  0.02293371,  0.00446185,  0.04841976,
         0.00746298,  0.04423537, -0.01965555, -0.02958047,  0.0137738 ],
       [ 0.03354345, -0.01383114, -0.01749373,  0.00576716, -0.00325754,
         0.0032355 ,  0.04443578, -0.01131739, -0.0082487 ,  0.02331621],
       [ 0.05156073,  0.00720141,  0.01553649, -0.01488   ,  0.04254658,
         0.01541946,  0.05828768, -0.02861129, -0.04046905,  0.01433513],
       [ 0.0414235 , -0.01066572,  0.00533527, -0.02395561,  0.04568579,
         0.00993827,  0.05862336, -0.01847712, -0.04673702,  0.02249537],
       [ 0.03303693,  0.01511562, -0.00361795, -0.027462  ,  0.02465398,
         0.02000896,  0.05029019, -0.02054171, -0.03193081,  0.02003735],
       [ 0.03303693,  0.01511563, -0.00361795, -0.027462  ,  0.02465398,
         0.02000896,  0.05029019, -0.02054171

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

_totalNormalSize = len(allNormalTextArray)
_totalAdSize = len(allAdTextArray)
actualNormalSize = _totalNormalSize
actualAdSize = _totalAdSize

y = np.concatenate((np.zeros(_totalNormalSize), np.ones(_totalAdSize)), axis=0)
#y = (np.zeros(len(allNormalTextArray)), np.ones(len(allAdTextArray)))

_i = 0
docVector = []

for v in vector:
    mean = np.mean(v)

    if mean < 0.00000001:
        if _i >= _totalNormalSize:
            actualAdSize -= 1
        else:
            actualNormalSize -= 1
    else:
        docVector.append(v)

    _i += 1

print(len(vector))
print(_totalNormalSize, _totalAdSize)
print(actualNormalSize, actualAdSize)
print(len(docVector))

1479
1105 374
931 213
1144


In [45]:
y = np.concatenate((np.zeros(_totalNormalSize), np.ones(_totalAdSize)), axis=0)
#y = (np.zeros(len(allNormalTextArray)), np.ones(len(allAdTextArray)))

X_train, X_test, y_train, y_test = train_test_split(vector, y, test_size=0.30, random_state=100)

svmModel = SVC(probability=True)  # 使用RBF核
svmModelRes = svmModel.fit(X_train, y_train)
print(svmModelRes.score(X_test, y_test))

svmModelRes.predict_proba()

0.7927927927927928


TypeError: _predict_proba() missing 1 required positional argument: 'X'

In [166]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

# y = trainer.train_model()
y = np.concatenate((np.zeros(len(allNormalTextArray)), np.ones(len(allAdTextArray))), axis=0)

print(len(y))
print(len(vector))

clf = MultinomialNB(alpha=1, fit_prior=True)
clf.fit(vector, y)

result = pd.DataFrame(vector)
keys = []
values = []

for key, value in vector:
    keys.append(key)
    values.append(value)

df = pd.DataFrame(data={"key": keys, "value": values})
colnames = df.sort_values("value")["key"].values
result.columns = colnames

#X_train, X_test, y_train, y_test = train_test_split(textMatrix, y, test_size=0.10, random_state=100)
print("Test Acc", clf.score(X_test, y_test))

1479
1479


ValueError: Negative values in data passed to MultinomialNB (input X)

In [167]:
import sklearn
sklearn.__version__

'0.22.1'